# Example 2: Operation of dipole moment operator to H2O vibrational ground state

| run type | restart | wavefunction | backend | Basis  | max iteration |
| ---       | ---   | ---          | ---     | ---    | ---   |
| operation  | True (file suffix `_gs`)  | MPS-SM (restart) | Numpy   | HO-FBR | 10    |

## 1. Import modules

- Required in **any** calculations

In [1]:
from pytdscf import BasInfo, Model, Simulator

## 2. Set FBR primitive basis

**MPS-MCTDH wavefunction**
$$
|\Psi_{\rm{MPS-MCTDH}}\rangle = \sum_{\mathbf \{j\}}\sum_{\mathbf \{\tau\}}
a\substack{j_1 \\ 1\tau_1}a\substack{j_2 \\ \tau_1\tau_2} \cdots a\substack{j_f \\ \tau_{f-1}1}
|\varphi_{j_1}^{(1)}(q_1)\rangle|\varphi_{j_2}^{(2)}(q_2)\rangle
\cdots|\varphi_{j_f}^{(f)}(q_f)\rangle
$$
where SPF is 
$$
\varphi_{j_p}^{(p)}(q_p) = \sum_{i_p=1}^{n_p} c_{i_p}^{j_p}\chi_{i_p}^{(p)}(q_p) \; (j_p = 1,2,\ldots, N_p)
$$

Here, select $\{\chi_{i_p}^{(p)}(q_p)\}$ as Harmonic Oscillator eigenfunction.
See detail in [documenation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.primints_cls.PrimBas_HO).
Here one define $n_p$ = 9, $N_p$ = 9. 

**NOTE**

- First argument of `Primbas_HO` is displaced dimensionless coordinate from q=0.0.

- In MPS,  $n = N$ (SM) is usually better than $n < M$ (MCTDH). Only when using a laptop computer, MCTDH sometimes works better. (because the required RAM in MCTDH is smaller than SM.)

- Primitive basis information must be the same as restart one.

In [2]:
from math import sqrt

from discvar import PrimBas_HO

from pytdscf import units
from pytdscf.potentials.h2o_potential import k_orig

freqs = [
    sqrt(k_orig[(1, 1)]),
    sqrt(k_orig[(2, 2)]),
    sqrt(k_orig[(3, 3)]),
]  # a.u. (sqrt{omega^2} = omega)
nprims = [9, 9, 9]
prim_info = [
    [
        PrimBas_HO(0.0, omega * units.au_in_cm1, nprim)
        for nprim, omega in zip(nprims, freqs, strict=False)
    ]
]  # number of state is 1 --> S0
nstate = len(prim_info)
ndof = len(prim_info[0])
basinfo = BasInfo(prim_info)

## 3. Set Dipole Operator (Polynomial Function)

Here, one use pre-calculated Polyonimal PES and DMS.
When `potential_emu=None`, `read_potential_nMR` does **not** inculde kinetic term.

In [3]:
from pytdscf.hamiltonian_cls import read_potential_nMR
from pytdscf.potentials.h2o_dipole import mu

mu  # Dipole has (x,y,z) elements

{(): [-1.69908e-15, 1.24913e-14, -1.93795],
 (3,): [-2.20831e-17, 0.00853527, -8.32759e-16],
 (2,): [1.50857e-17, 2.08217e-15, -0.00326347],
 (1,): [6.37588e-18, 8.65662e-16, 0.0142383],
 (3, 3): [3.5274e-18, -1.35302e-15, -2.31565e-05],
 (2, 3): [3.46044e-18, -0.000294259, -7.3169e-16],
 (2, 2): [-1.5306e-18, -1.42229e-15, 0.00020955],
 (1, 3): [1.45261e-17, 0.000327409, -2.99469e-17],
 (1, 2): [3.90656e-18, 1.26166e-16, -0.000112968],
 (1, 1): [-6.45481e-18, 6.79098e-16, 0.000192831],
 (3, 3, 3): [-1.34711e-21, 7.33335e-06, 9.41511e-22],
 (2, 3, 3): [2.2067e-22, -3.92968e-22, 3.0608e-06],
 (1, 3, 3): [-2.55725e-22, 4.55392e-22, -3.54702e-06],
 (2, 2, 3): [6.16547e-22, -3.35633e-06, -4.3091e-22],
 (2, 2, 2): [1.69378e-22, -3.01627e-22, 2.34936e-06],
 (1, 2, 2): [3.17065e-22, -5.64628e-22, 4.39783e-06],
 (1, 1, 3): [-1.08836e-21, 5.92476e-06, 7.60666e-22],
 (1, 1, 2): [-2.92033e-23, 5.20049e-23, -4.05062e-07],
 (1, 1, 1): [5.60185e-22, -9.97572e-22, 7.77e-06]}

In [4]:
dipole = read_potential_nMR(potential_emu=None, cut_off=-1.0e-10, dipole_emu=mu)
print("Constant terms:", dipole.coupleJ[0][0])
print("Onesite terms:", *dipole.onesite[0][0])
print("Multisite terms", *dipole.general[0][0])
operators = {"hamiltonian": dipole}

Constant terms: (-1.9379499999999892+0j)
Onesite terms: +8.5353e-03 q^1_2 -3.2635e-03 q^1_1 +1.4238e-02 q^1_0 -1.1578e-05 q^2_2 +1.0477e-04 q^2_1 +9.6416e-05 q^2_0 +1.2222e-06 q^3_2 +3.9156e-07 q^3_1 +1.2950e-06 q^3_0
Multisite terms -2.9426e-04 q^1_1 * q^1_2 +3.2741e-04 q^1_0 * q^1_2 -1.1297e-04 q^1_0 * q^1_1 +1.5304e-06 q^1_1 * q^2_2 -1.7735e-06 q^1_0 * q^2_2 -1.6782e-06 q^2_1 * q^1_2 +2.1989e-06 q^1_0 * q^2_1 +2.9624e-06 q^2_0 * q^1_2 -2.0253e-07 q^2_0 * q^1_1


## 4. Set wave function (MPS) and All Model

- `m_aux_max` is MPS bond dimenison (maximum of auxiliary index $\tau_p$)

In [5]:
model = Model(basinfo=basinfo, operators=operators)
model.m_aux_max = 9

## 5. Execute Calculation

In [6]:
! ls wf_h2o_polynomial_gs.pkl

wf_h2o_polynomial_gs.pkl


F.Y.I., See also about [Simulator](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.simulator_cls.Simulator)

This run type prepare $|\Psi_{\mu}\rangle$ by variationally optimizing
$$\langle \delta\Psi_{\mu}|\Psi_{\mu} - \hat{\mu}\Psi_{\mathrm{gs}}\rangle=0$$
where $\Psi_{\mathrm{gs}}$ is a vibrational ground state wavefunction.

**NOTE**

- Runtype cannnot rebind. If you want to change runtype, you should restart kernel.

In [7]:
jobname = "h2o_polynomial"
simulator = Simulator(jobname, model, backend="numpy", verbose=4)
simulator.operate(
    loadfile_ext="_gs", savefile_ext="_dipole", restart=True, maxstep=10
)

2025-02-04 23:34:41,780 - INFO:main.pytdscf._const_cls - 
     ____     __________   .____ ____   _____
    / _  |   /__  __/ _ \ / ___ / _  \ / ___/
   / /_) /_  __/ / / / ||/ /__ / / )_// /__
  /  ___/ / / / / / / / |.__  / |  __/ ___/
 /  /  / /_/ / / / /_/ /___/ /| \_/ / /
/__/   \__, /_/ /_____/_____/ \____/_/
      /____/

2025-02-04 23:34:41,781 - INFO:main.pytdscf._const_cls - Log file is ./h2o_polynomial_operate/main.log
2025-02-04 23:34:41,782 - INFO:main.pytdscf.simulator_cls - Set integral of FBR basis
2025-02-04 23:34:41,805 - INFO:main.pytdscf.simulator_cls - Set initial wave function (FBR basis)
2025-02-04 23:34:41,807 - INFO:main.pytdscf.simulator_cls - Wave function is loaded from wf_h2o_polynomial_gs.pkl
2025-02-04 23:34:41,807 - INFO:main.pytdscf.simulator_cls - Start: apply operator to wave function
2025-02-04 23:34:41,833 - INFO:main.pytdscf.simulator_cls - Wave function is saved in wf_h2o_polynomial_dipole.pkl
2025-02-04 23:34:41,833 - INFO:main.pytdscf.simulator_

(1.9386775043559428, <pytdscf.wavefunction.WFunc at 0x10dc04f50>)

## 6. Check Log file
See `h2o_polynomial_operate/main.log`, which is defined by `jobname`.

In [8]:
!tail h2o_polynomial_operate/main.log

Wave function is loaded from wf_h2o_polynomial_gs.pkl
Start: apply operator to wave function
----------------------------------------
iterations: 0 norm: 1.9386775043559425
convergence : 0.9977392513409664
----------------------------------------
iterations: 1 norm: 1.9386775043559428
convergence : 1.0000000000000004
Wave function is saved in wf_h2o_polynomial_dipole.pkl
End  : apply operator to wave function
